## 数据预处理过程

### 1. 对审计公司的名称进行更改

Co., Ltd是责任有限公司和股份有限公司的缩写

处理出现的问题：

BDO China Shu Lun Pan Certified Public Accountants LLP

BDO CHINA SHU LUN PAN Certified Public Accountants LLP (Special General Partnership)

只是公司名称大小写的区别，应该还是同一家公司

### 2. 建立哑变量

八大记为1，非八大记为0

Standard Unqualified Opinion

Unqualified Opinion with Emphasis of Matter Paragraph

标准记为1，非标准记为0

### 3. 排序

原本是按照时间排序，现在按照公司名排序，然后再排时间

### 4. 生成时间

real可以将字符串转化为数字

substr(str,n1,n2)提取字符串，n1代表起始位置，n2代表长度

剔除半年报数据，仅留下年报数据

### 5. 根据公司、年份进行分组

_N代表样本数

_n代表样本的序号

样本数等于1记为0，不等于1则记为样本序号，去掉序号大于1的样本

### 6. 保留合并报表数据以及年报数据

statement type = A表示合并报表数据

### 7. 计算财务比率

Debt_to_Equity = TotalLiabilities/TotalShareholdersEquity

Current_ratio = TotalCurrentAssets/TotalAssets

## 数据分析

In [1]:
import pandas as pd

In [2]:
df = pd.read_stata(r'D:/WORK SPACE/Siyu Lin/Data & Codes/Audit_Combas_data.dta')

df

,Stkcd,Accper,TotalAssets,year,Debt_to_Equity,Current_ratio,TotalAuditFees,big8,Audit_Opinion,Stkcd1
0,000001,2011-12-31,1.258177e+12,2011.0,15.690995,NaN,8190000.0,1.0,1.0,1.0
1,000001,2012-12-31,1.606537e+12,2012.0,17.945283,0.000000,11320000.0,1.0,1.0,1.0
2,000001,2013-12-31,1.891741e+12,2013.0,15.878338,0.000000,9580000.0,1.0,1.0,1.0
3,000001,2014-12-31,2.186459e+12,2014.0,15.697027,0.000000,9676000.0,1.0,1.0,1.0
4,000001,2015-12-31,2.507149e+12,2015.0,14.524142,0.000000,10066000.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
16015,900957,2012-12-31,3.914345e+08,2012.0,0.025788,0.030261,350000.0,0.0,1.0,900957.0
16016,900957,2013-12-31,3.954345e+08,2013.0,0.016397,0.184145,350000.0,0.0,1.0,900957.0
16017,900957,2014-12-31,8.872896e+08,2014.0,1.178945,0.120545,350000.0,0.0,1.0,900957.0
16018,900957,2015-12-31,1.015625e+09,2015.0,1.473041,0.147454,350000.0,1.0,1.0,900957.0


### Step1

In [3]:
df_des = df.loc[:,['TotalAssets','Debt_to_Equity','Current_ratio',
                   'TotalAuditFees','big8','Audit_Opinion']]

df_des.describe()

,TotalAssets,Debt_to_Equity,Current_ratio,TotalAuditFees,big8,Audit_Opinion
count,1.602000e+04,16020.000000,15928.000000,1.566000e+04,16020.000000,16020.000000
mean,5.500622e+10,1.407971,0.568546,1.626832e+06,0.326342,0.962484
std,7.038368e+11,7.773084,0.222327,7.270127e+06,0.468889,0.190027
min,3.083701e+06,-340.170598,0.000000,1.000000e+04,0.000000,0.000000
25%,1.370392e+09,0.331963,0.417729,5.200000e+05,0.000000,1.000000
50%,2.986036e+09,0.724614,0.589101,7.500000e+05,0.000000,1.000000
75%,7.606772e+09,1.527468,0.741840,1.200000e+06,1.000000,1.000000
max,2.413726e+13,531.924648,1.000000,2.220000e+08,1.000000,1.000000


### Step2

In [4]:
from scipy import stats

In [5]:
# 对缺失值进行剔除处理
AF_big8 = df.loc[df['big8'] == 1, 'TotalAuditFees'].dropna(axis=0, how='any')
AF_nonbig8 = df.loc[df['big8'] == 0, 'TotalAuditFees'].dropna(axis=0, how='any')

In [6]:
# 检验是否AF_big8和AF_nonbig8是否具有方差齐性。若pvalue<0.05则认为不具有方差齐性
stats.levene(AF_big8, AF_nonbig8)

LeveneResult(statistic=235.79507601281244, pvalue=7.880265919554605e-53)

In [7]:
stats.ttest_ind(AF_big8, AF_nonbig8, equal_var = False)

Ttest_indResult(statistic=11.327609333610312, pvalue=2.1286320014234735e-29)

在置信度为95%的情况下，拒绝原假设。审计公司不同，审计费用有显著性差异

### Step3

In [8]:
TA_Q1 = df['TotalAssets'].quantile(0.25)
TA_Q2 = df['TotalAssets'].quantile(0.5)
TA_Q3 = df['TotalAssets'].quantile(0.75)

AF1 = df.loc[df['TotalAssets'] < TA_Q1, 'TotalAuditFees'].dropna(axis=0, how='any')
AF2 = df.loc[(df['TotalAssets'] >= TA_Q1) & (df['TotalAssets'] < TA_Q2), 'TotalAuditFees'].dropna(axis=0, how='any')
AF3 = df.loc[(df['TotalAssets'] >= TA_Q2) & (df['TotalAssets'] < TA_Q3), 'TotalAuditFees'].dropna(axis=0, how='any')
AF4 = df.loc[df['TotalAssets'] >= TA_Q3, 'TotalAuditFees'].dropna(axis=0, how='any')

In [9]:
stats.f_oneway(AF1, AF2, AF3, AF4)

F_onewayResult(statistic=239.4881583952804, pvalue=6.057930130597904e-152)

在置信度为95%的情况下，拒绝原假设。公司规模不同，审计费用有显著性差异

### Step4

In [10]:
import statsmodels.formula.api as smf

In [15]:
# hetero
results_LR_het = smf.ols("TotalAuditFees ~ big8", data = df).fit(cov_type='HC0')

results_LR_het.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         TotalAuditFees   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     128.3
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           1.23e-29
Time:                        22:02:32   Log-Likelihood:            -2.6951e+05
No. Observations:               15660   AIC:                         5.390e+05
Df Residuals:                   15658   BIC:                         5.390e+05
Df Model:                           1                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   9.796e+05   1.26e+04     78.024      0.000    9.55e+05       1e+06
big8        1.988e+06   1.76e+05     11.329      0.000    1.64e+06    2.33e+06
==============================================================================
Omnibus:                    34071.910   Durbin-Watson:                   0.334
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        147126984.484
Skew:                          19.884   Prob(JB):                         0.00
Kurtosis:                     476.181   Cond. No.                         2.41
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [16]:
# cluster has some error
# results_LR_clu = smf.ols("TotalAuditFees ~ big8", data = df).fit(cov_type='cluster', cov_kwds={'groups': df['Stkcd1']})

# results_LR_clu.summary()

### Step5

In [13]:
# hetero
results_MLR_het = smf.ols("TotalAuditFees ~ big8 + TotalAssets + Debt_to_Equity + Current_ratio + Audit_Opinion", data = df).fit(cov_type='HC0')

results_MLR_het.summary()

D:\Anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 4
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         TotalAuditFees   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     47.39
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           1.16e-39
Time:                        20:38:52   Log-Likelihood:            -2.5449e+05
No. Observations:               15568   AIC:                         5.090e+05
Df Residuals:                   15562   BIC:                         5.090e+05
Df Model:                           5                                         
Covariance Type:                  HC0                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       8.484e+05    7.4e+04     11.470      0.000    7.03e+05    9.93e+05
big8            7.014e+05   5.75e+04     12.209      0.000    5.89e+05    8.14e+05
TotalAssets     9.153e-06   5.38e-07     17.001      0.000     8.1e-06    1.02e-05
Debt_to_Equity -3364.7568   2856.270     -1.178      0.239   -8962.943    2233.429
Current_ratio  -3.897e+05   1.42e+05     -2.741      0.006   -6.68e+05   -1.11e+05
Audit_Opinion   2.916e+05   3.77e+04      7.728      0.000    2.18e+05    3.66e+05
==============================================================================
Omnibus:                    22228.819   Durbin-Watson:                   0.578
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         27440009.073
Skew:                           7.963   Prob(JB):                         0.00
Kurtosis:                     208.057   Cond. No.                     4.40e+12
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 4.4e+12. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
# cluster has some error
# results_MLR_clu = smf.ols("TotalAuditFees ~ big8 + TotalAssets + Debt_to_Equity + Current_ratio + Audit_Opinion", data = df).fit(cov_type='cluster', cov_kwds={'groups': df['Stkcd']})

# results_MLR_clu.summary()